<h2>Project 8: Deep Learning</h2>


<h3>Introduction</h3>
In this project, you will implement a neural network.
We broke it apart into several functions and a pre-processing step.</p>

  <p> 
  An important step to make neural networks function properly is to scale the data properly. Therefore, first implement the preprocess function
  It takes as input the training and the test data and transforms them both with the same mapping: $\vec x_i\rightarrow U\vec x_i-\vec{m}$. 
  After this transformation the training data set should have zero-mean and each feature should have standard deviation 1 . The same transformation is then also applied to the test data (Hint: The matrix U is typically diagonal.) 
  <!--	 <li>HINT 2: Ideally you would like the input features to be de-correlated. The correlation matrix should be diagonal (in this case even the identity matrix). One way to do this is to project the data onto the PCA principal components (which we will still cover later in the course). You can get the transposed projection matrix by calling $pcacov(xTr')$. Make sure to apply PCA <i>after</i> you subtracted off the mean. </li>-->
  </p>

In [4]:
using MAT
#<GRADED>
function preprocess(xTr,xTe)
# function [xTr,xTe,u,m]=preprocess(xTr,xTe);
#
# Preproces the data to make the training features have zero-mean and
# standard-deviation 1
#
# output:
# xTr - pre-processed training data 
# xTe - pre-processed testing data
# 
# u,m - any other data should be pre-processed by x-> u*(x-m)
#

    Ntr,D=size(xTr);
    Nte,D=size(xTe);

    ##  Todo 1
    
    return (xTr,xTe,u,m)
end;
#</GRADED>

preprocess (generic function with 1 method)

<p>We need to define some transition functions and their gradients. Implement the sigmoid, tanh, and ReLU transition functions (the $sin()$ transition function is already implemented for you).</p> 
 <ol>
     <li>ReLU: $\sigma(z) = \max(0, z)$</li>
     <li>Sigmoid: $\sigma(z) = \frac{1}{1 + e^{-z}}$</li>
     <li>tanh: $\sigma(z) = \frac{e^z - e^{-z}}{e^z + e^{-z}}$</li>
 </ol>

In [5]:
#<GRADED>
function get_transition_func(transtype)
#	Given the type, gets a specific transition function
#   INPUT
#   type "sigmoid", "tanh", "ReLU"
#   OUTPUT
#   trans_func transition function (function)
#   trans_func_der derivative of the transition function (function)

    # type must be one of the defined transition functions            
    assert(transtype in ["sigmoid", "tanh", "ReLU","sin"])
            
    if transtype=="sin"      
            trans_func=z->sin(z);  
            trans_func_der=z->cos(z);                                      
    end

    ## Todo 2
    
    return (trans_func,trans_func_der)
end
#</GRADED>

get_transition_func (generic function with 1 method)

<p>If everything is correct we can now plot these functions and check their gradients. The gradient errors should all be very small (less than $10^{-10}$.) </p>

In [3]:
f, delta_f = get_transition_func("sin")

(#1,#2)

In [6]:
using PyPlot

function numericalgradient(fun,x,e);
    dh=0;
    nx=x;    # copy the weight vector
    nx+=e;  # perturn dimension i
    l1=fun(nx); # compute loss
    nx-=2*e; # perturm dimension i again
    l2=fun(nx); # compute loss
    dh=(l1[1]-l2[1])/(2*e); # the gradient is the slope of the loss
    return(dh);
end;


figure()
names=["sigmoid","tanh","ReLU","sin"]
for i=1:4
    name=names[i];
    subplot(1,4,i)
    f,δf=get_transition_func(name)
    x=-5:0.1:5;
    plot(x,f(x))
    title(name)
    grid(true);
    axis([-5,5,-1,1])
    
    
    print("$(name) gradient check at x=1")
    dh=numericalgradient(f,1,1e-5);
    dy=δf(1);
    graderror=norm(dh-dy)/norm(dh+dy)
    if graderror<1e-10; print("✔");else; print("✘")end;

    print(" at x=-1")
    dh2=numericalgradient(f,-1,1e-5);
    dy2=δf(-1);
    graderror+=norm(dh2-dy2)/norm(dh2+dy2)
    if graderror<1e-10; print("✔");else; print("✘")end;
    print("\n");
 end;

LoadError: LoadError: UndefVarError: trans_func not defined
while loading In[6], in expression starting on line 17


  <p> Now implement the forward pass function
  <pre>forward_pass(W,xTr,trans_func)</pre>
  It takes the weights for the network, the training data, and the transition function to be used between layers.  It should output the result at each node for the forward pass.  $W\{1\}$ stores the weights for the last layer of the network and $W\{end\}$ the <i>first</i> layer.
  </p>
 


In [5]:
function initweights(specs)
    weights=[]
    for i=1:length(specs)-1
        append!(weights,[randn(specs[i],specs[i+1])])        
    end;    
    return(weights)
end;    

In [6]:
#<GRADED>
function forward_pass(weights, xTr, trans_func )
# function forward_pass(weights,xTr,trans_func)
# 
# INPUT:
# weights weights (cell array)
# xTr nxd matrix (each column is an input vector)
# trans_func transition function to apply for inner layers
#
# OUTPUTS:
# as = result of forward pass 
# zs = result of forward pass (zs[end] output layer of the forward pass) 
#

    # Todo 3
    return(as,zs)
end;
#</GRADED>

forward_pass (generic function with 1 method)

In [7]:
weights=initweights([2,20,20,1])
f,δf=get_transition_func("sigmoid")
x=rand(100,2);
as,zs=forward_pass(weights,x,f)

LoadError: LoadError: UndefVarError: initweights not defined
while loading In[7], in expression starting on line 1

 <p> Now compute the loss for the network
  <pre>compute_loss(zs, yTr)</pre>
  It takes the output of the forward pass and the training labels.  It should compute the loss for the entire training set averaging over all the points:
  $$L(x, y) = \frac{1}{2n}(H(x) - y)^2$$
  </p>

In [8]:
#<GRADED>
function compute_loss(zs, yTr)
# function [loss] = compute_loss(zs, yTr)
#
# INPUT:
# zs output of forward pass (cell array)
# yTr 1xn matrix (each entry is a label)
#
# OUTPUTS:
# 
# loss = the average squared loss obtained with zs and yTr
#
#
    delta=zs[end][:]-yTr[:];
    n=length(yTr);
    loss=0;
    
    ## Todo 4
    
    return(loss)
end;
#</GRADED>

yTr=rand(100);
compute_loss(zs,yTr)

LoadError: LoadError: UndefVarError: zs not defined
while loading In[8], in expression starting on line 25

<p> Now implement <i>back propagation</i> 
  <pre>backprop(W, as, zs, yTr,  der_trans_func)</pre>
  to compute the gradient for the weights and bias terms. 
  It takes the weights for the network, the outputs of the forward pass, the training labels, and the derivative of the transition function.  Use the chain rule to calculate the gradient of the weights.
  </p>
  		



In [9]:
#<GRADED>
function  backprop(weights, as,zs, yTr,  δf)
# function [gradient] = backprop(W, as, zs, yTr,  der_trans_func)
#
# INPUT:
# W weights (cell array)
# as output of forward pass (cell array)
# zs output of forward pass (cell array)
# yTr 1xn matrix (each entry is a label)
# der_trans_func derivative of transition function to apply for inner layers
#
# OUTPUTS:
# 
# gradient = the gradient at w as a cell array of matrices
#
    n=length(yTr);
    δ=zs[end]-yTr;
    # compute gradient with back-prop
    gradients=copy(weights);    
    for i=length(weights):-1:1
        # Todo 5
    end; 
    return(gradients)
 end;
#</GRADED>

backprop (generic function with 1 method)

### Momentum

In [10]:
x=(0:0.1:5)[:]
y=(x.^2.+sin(x).*10)[:];
#y=3.*x;

x2d=[x x.*0.+1];

weights=initweights([2,200,1])
#weights[1]=3;
Momentum=copy(weights);
for j=1:length(weights)
   Momentum[j]*=0.0;
end;    


α=0.01;
M=10000;
β=0.8;
losses=zeros(M)
tic()
for i=1:M
    f,δf=get_transition_func("ReLU")
    as,zs=forward_pass(weights,x2d,f)
    losses[i]=compute_loss(zs,y)
    gradients=backprop(weights,as,zs,y,δf)
    for j=1:length(weights)
        Momentum[j]=Momentum[j].*β+(1-β).*gradients[j];
        weights[j]-=α.*Momentum[j];
    end;    
end;
print(toc())
subplot(1,2,1);
plot(x,y,"b");
plot(x,zs[end],"r")
subplot(1,2,2);
semilogy(losses)

LoadError: LoadError: UndefVarError: initweights not defined
while loading In[10], in expression starting on line 7

### Adagrad

In [ ]:
x=(0:0.1:5)[:]
y=(x.^2.+sin(x).*10)[:];
#y=3.*x;

x2d=[x x.*0.+1];

weights=initweights([2,200,1])


hist_grad=copy(weights);
for j=1:length(weights)
    hist_grad[j]*=0.0;
end; 


α=0.01;
M=10000;
β=0.8;
eps = 1e-6

losses=zeros(M)
tic()
for i=1:M
    f,δf=get_transition_func("ReLU")
    as,zs=forward_pass(weights,x2d,f)
    losses[i]=compute_loss(zs,y)
    gradients=backprop(weights,as,zs,y,δf)
    for j=1:length(weights)
        hist_grad[j] += gradients[j].^2
        adj_grad = gradients[j] ./ (eps + hist_grad[j].^0.5)
        weights[j]-=α.*adj_grad;
    end;    
end;
print(toc())
subplot(1,2,1);
plot(x,y,"b");
plot(x,zs[end],"r")
subplot(1,2,2);
semilogy(losses)

### Custom step-size decrease schedule

In [ ]:
x=(0:0.1:5)[:]
y=(x.^2.+sin(x).*10)[:];
#y=3.*x;

x2d=[x x.*0.+1];

weights=initweights([2,200,1])


hist_grad=copy(weights);
for j=1:length(weights)
    hist_grad[j]*=0.0;
end; 


α=0.01;
M=10000;
β=0.8;
eps = 1e-6

losses=zeros(M)
tic()
for i=1:M
    f,δf=get_transition_func("ReLU")
    as,zs=forward_pass(weights,x2d,f)
    losses[i]=compute_loss(zs,y)
    gradients=backprop(weights,as,zs,y,δf)
    for j=1:length(weights)
        if i != 0
            hist_grad[j] = β * hist_grad[j] + (1 - β) * gradients[j].^2
        else
            hist_grad[j] = gradients[j].^2
        end
        adj_grad = gradients[j] ./ (eps + hist_grad[j].^0.5)
        weights[j]-=α.*adj_grad;
    end;    
end;
print(toc())
subplot(1,2,1);
plot(x,y,"b");
plot(x,zs[end],"r")
subplot(1,2,2);
semilogy(losses)